In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [3]:
caffe.set_mode_cpu()
net = caffe.Net('../models/swimmers_fullconv/swimmers_fullconv_small.prototxt', 
                '../models/swimmers_fullconv/out_new_small.binaryproto', caffe.TEST)
net.reshape()

In [4]:
# find max_kernels weight blob
max_kernels = net.params['idpr_dt'][0].data
# max_kernels.reshape(1, 152, 35, 35)
print max_kernels.shape

(1, 152, 7, 7)


In [5]:
# find scale term weight blob
scale_terms = net.params['idpr_dt'][1].data
print scale_terms.shape

(152,)


In [6]:
multiplier = 0.01
kernel = np.zeros((max_kernels.shape[2], max_kernels.shape[3]))
half_size = (kernel.shape[0] - 1) /2
tmp_sum = 0

for x in xrange(-half_size, half_size+1):
    for y in xrange(-half_size, half_size+1):
        pixel = multiplier * (x**2 + y**2)
        tmp_sum += pixel
        kernel[x + half_size ,y + half_size] = pixel

print kernel #* (1 / tmp_sum)

[[ 0.18  0.13  0.1   0.09  0.1   0.13  0.18]
 [ 0.13  0.08  0.05  0.04  0.05  0.08  0.13]
 [ 0.1   0.05  0.02  0.01  0.02  0.05  0.1 ]
 [ 0.09  0.04  0.01  0.    0.01  0.04  0.09]
 [ 0.1   0.05  0.02  0.01  0.02  0.05  0.1 ]
 [ 0.13  0.08  0.05  0.04  0.05  0.08  0.13]
 [ 0.18  0.13  0.1   0.09  0.1   0.13  0.18]]


In [7]:
# write every kernel as the same!
for i in xrange(max_kernels.shape[1]):
    max_kernels[0, i] = kernel

In [8]:
# write scale terms to 1 (for now...)

for i in xrange(scale_terms.shape[0]):
    scale_terms[i] = 1

In [9]:
print max_kernels, scale_terms

[[[[ 0.18000001  0.13        0.1        ...,  0.1         0.13        0.18000001]
   [ 0.13        0.08        0.05       ...,  0.05        0.08        0.13      ]
   [ 0.1         0.05        0.02       ...,  0.02        0.05        0.1       ]
   ..., 
   [ 0.1         0.05        0.02       ...,  0.02        0.05        0.1       ]
   [ 0.13        0.08        0.05       ...,  0.05        0.08        0.13      ]
   [ 0.18000001  0.13        0.1        ...,  0.1         0.13        0.18000001]]

  [[ 0.18000001  0.13        0.1        ...,  0.1         0.13        0.18000001]
   [ 0.13        0.08        0.05       ...,  0.05        0.08        0.13      ]
   [ 0.1         0.05        0.02       ...,  0.02        0.05        0.1       ]
   ..., 
   [ 0.1         0.05        0.02       ...,  0.02        0.05        0.1       ]
   [ 0.13        0.08        0.05       ...,  0.05        0.08        0.13      ]
   [ 0.18000001  0.13        0.1        ...,  0.1         0.13        0.180000

In [10]:
# save changes to new binaryproto
net.save('../models/swimmers_fullconv/out_new_small.binaryproto')